<a href="https://colab.research.google.com/github/adventHymnals/resources/blob/master/Sheets2Resume.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sheets2Resume
- Generate a professional resume from data in a spreadsheet.
- Focus on the content and we'll take care of of the formatting and appearance.
- Before you proceed, please note that this system will only work for single page resumes, for londer resumes, you can check out, in order of usefulness:
1. [This resume creator](https://csymapp.github.io/atsresume/) whose code  is [on github](https://github.com/csymapp/atsresume)
2. [this resume creator](https://jsoncv.reorx.com/editor/#root.languages). *This one will not help you with ATS*

Then use [this ATS tool](https://www.resumego.net/resume-checker/) to check the score of your resume.

In [ ]:
#@title Select sheet with cv
#%%capture
!apt install xattr &>log

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!mkdir -p /content/drive/MyDrive/resume_v1/public/
!mkdir -p /content/drive/MyDrive/resume_v1/private/


import os
from IPython.display import HTML
from subprocess import getoutput
sheetType=""
## check if worksheet
###
message = None
output = !ls "/content/drive/MyDrive/resume_v1/public/resume_v1.gsheet"
if "No such file or directory" in output[0]: # no gsheet, look for xlsx
    output = !ls "/content/drive/MyDrive/resume_v1/public/resume_v1.xlsx"
    if "No such file or directory" in output[0]: # no xlsx, create it
        !wget https://github.com/csymapp/vue-resume/raw/master/resume_v1.xlsx -O "/content/drive/MyDrive/resume_v1/public/resume_v1.xlsx" > /dev/null 2>&1
        sheetId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public/resume_v1.xlsx")
        file_url = f"https://docs.google.com/spreadsheets/d/{sheetId}/edit"
        dirId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public")
        message = f'resume_v1 sheet downloaded and saved locally in <a target="_blank" href="https://drive.google.com/drive/u/0/folders/{dirId}">public</a>.'
    else:
        sheetId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public/resume_v1.xlsx")
        file_url = f"https://docs.google.com/spreadsheets/d/{sheetId}/edit"
        message = f'Excel sheet <a target="_blank" href="{file_url}">resume_v1</a> found.'
    sheetType = "xlsx"
    os.environ['SHEETID'] = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public/resume_v1.xlsx")
    os.environ['SHEETPATH'] = "/content/drive/MyDrive/resume_v1/public/resume_v1.xlsx"
else:
    sheetId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public/resume_v1.gsheet")
    file_url = f"https://docs.google.com/spreadsheets/d/{sheetId}/edit"
    message = f'Google sheet <a target="_blank" href="{file_url}">resume_v1</a> found.'
    sheetType = "gsheet"
    os.environ['SHEETID'] = sheetId
    os.environ['SHEETPATH'] = "/content/drive/MyDrive/resume_v1/public/resume_v1.gsheet"
display(HTML(message))
os.environ['SHEETTYPE'] = sheetType ## Where your resume data is stored
privateId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/private")
os.environ['PRIVATEID'] = privateId ## Where your resume data is stored

### convert the sheet data to json
import os

def rows_to_json(rows, title):
    if len(rows) < 2:
        return "Error: Not enough rows to create JSON"

    json_data = {}

    header = rows[0]
    for row in rows[1:]:
        key = str(row[0])
        key_lower = key.lower()
        key_to_use = key if str(title).lower().endswith("_skills") or str(title).lower() == "education" or str(title).lower() == "work" or str(title).lower() == "projects" or str(title).lower() == "work_tags" else key_lower
        #key_to_use = key # override previous
        json_data[key_to_use] = {}

        for i, value in enumerate(row[1:]):
            header_value = str(header[i+1])
            #header_value_lower = header_value.lower()
            #key_to_use1 = header_value if header_value_lower.endswith("_skills") or str(title).lower() == "education" else header_value_lower
            key_to_use1 = header_value # override previous
            if pd.isna(value):
                json_data[key_to_use][key_to_use1] = ""
            else:
                json_data[key_to_use][key_to_use1] = value

    return json_data

all_json_data = {}

from datetime import datetime
def convert_timestamps_to_datetime(data):
    for entry in data.values():
        for key, value in entry.items():
            if isinstance(value, pd.Timestamp):
                entry[key] = value.strftime('%Y-%m-%d')
            if isinstance(value, datetime):
                entry[key] = value.strftime('%Y-%m-%d')


if os.environ.get('SHEETTYPE') == 'xlsx':
    !pip install pandas > /dev/null 2>&1
    import pandas as pd
    excel_file_path = '/content/drive/MyDrive/resume_v1/public/resume_v1.xlsx'
    # Read all sheets in the Excel file into a dictionary of DataFrames
    all_sheets = pd.read_excel(excel_file_path, sheet_name=None)
    for sheet_name, df in all_sheets.items():
        header_row = df.columns.tolist()
        rows = [header_row] + df.values.tolist()
        json_data = rows_to_json(rows, sheet_name)
        convert_timestamps_to_datetime(json_data)
        #print(json_data)
        all_json_data[sheet_name] = json_data
else:
    from google.colab import auth
    auth.authenticate_user()

    import gspread
    from google.auth import default
    creds, _ = default()

    gc = gspread.authorize(creds)

    spreadsheet_list = gc.openall()
    worksheet = None
    # Print the titles of all spreadsheets
    for spreadsheet in spreadsheet_list:
        if spreadsheet.title == "resume_v1":
          if os.environ.get('SHEETID') == spreadsheet.id: # if there are multiple sheets with the same name, ensure to pick the one with the correct id
            worksheet = spreadsheet


    #worksheet = gc.open("resume_v1")
    sheet_list = worksheet.worksheets()
    for sheet in sheet_list:
        rows = sheet.get_all_values()
        json_data = rows_to_json(rows, sheet.title)
        all_json_data[sheet.title] = json_data

import json
#print(all_json_data)
main_json = json.dumps(all_json_data, indent=4)

###
### Reorder "basics"
###
# Load the JSON data
data = json.loads(main_json)
#for key, value in data['basics'].items():
    #data['basics'][key] = value['value']
for key, value in data['basics'].items():
    for key1, value1 in data['basics'][key].items():
      if key1.lower() == 'value':
          data['basics'][key] = value1
# Convert back to JSON


data['basics']['profiles'] = list(data['profiles'].values())
# Remove the original profiles key
data.pop('profiles')

#for key, value in data['work'].items():
    #data['work'][key] = value['value']
# Convert back to JSON

for position, work_entry in data["work"].items():
    work_entry["position"] = position
for position, work_entry in data["work"].items():
    work_entry["company"] = work_entry.pop("institution")
    work_entry["startDate"] = work_entry.pop("start")
    work_entry["endDate"] = work_entry.pop("end")
# Iterate through each entry in the "work" object and combine descriptions
for work_entry in data["work"].values():
    highlights = [work_entry.pop(key) for key in list(work_entry.keys()) if key.startswith("description_") and work_entry[key]]
    work_entry["highlights"] = highlights

data['work'] = list(data['work'].values())


### projects
for project, project_entry in data["projects"].items():
    project_entry["name"] = project
for project_entry in data["projects"].values():
    highlights = [project_entry.pop(key) for key in list(project_entry.keys()) if key.startswith("description_")  and project_entry[key]]
    project_entry["highlights"] = highlights
for position, project_entry in data["projects"].items():
    project_entry["startDate"] = project_entry.pop("start")
    project_entry["endDate"] = project_entry.pop("end")
# Iterate through each entry in the "projects" object and split the "keywords" string into an array
for project in data["projects"].values():
    keywords_str = project.get("keywords", "")
    keywords_array = [keyword.strip() for keyword in keywords_str.split(",")]
    project["keywords"] = keywords_array
data['projects'] = list(data['projects'].values())
#data.pop('profiles')

### education
for education, education_entry in data["education"].items():
    education_entry["area"] = education
for position, education_entry in data["education"].items():
    education_entry["startDate"] = education_entry.pop("start")
    education_entry["endDate"] = education_entry.pop("end")
data['education'] = list(data['education'].values())


### work_tags
for work_tag, work_tag_entry in data["work_tags"].items():
    work_tag_entry["display"] = work_tag
for tag_data in data["work_tags"].values():
    if tag_data["website"] == "":
        tag_data.pop("website")


for work_entry in data["work"]:
    tags_str = work_entry.get("tags", "")
    tags_array = [tag.strip() for tag in tags_str.split(",")]
    work_entry["tags"] = []

    # Pull value from "work_tags" for each item in "tags" and put objects in "tags"
    #for tag in tags_array:
    #    if tag.lower() in data["work_tags"]:
    #        tmp = data["work_tags"][tag.lower()]
    #        tmp["display"] = tag
    #        work_entry["tags"].append(tmp)
    for tag in tags_array:
      lower_tag = tag.lower()
      for work_tag, work_tag_value in data["work_tags"].items():
          if work_tag.lower() == lower_tag:
              tmp = work_tag_value
              tmp["display"] = tag
              work_entry["tags"].append(tmp)
#data['work_tags'] = list(data['work_tags'].values())

## skills
for key, value in data.items():
    if key.endswith("_skills") and isinstance(value, dict):
        for skill, skill_entry in value.items():
            skill_entry["display"] = skill

data["skills"] = {}

def convert_skill_to_keywords(skill):
    keywords = []
    if "display" in skill:
        keyword = {
            "display": skill["display"]
        }
        if "website" in skill:
            keyword["website"] = skill["website"]
        if "level" in skill:
            keyword["level"] = skill["level"]
        keywords.append(keyword)
    return keywords

# Create a new field called "keywords" for each skill category
#for category, skills in data.items():
    #if category.endswith("_skills"):
        #data["skills"][category] = {}
        #category_name = category.replace("_skills", "")
        #title = data["skill_configs"][category_name]["title"]
        #data["skills"][category]["name"] = title
        ##data["skills"][category]["keywords"] = skills
        #data["skills"][category]["keywords"] = list(skills.values())
for category, skills in data.items():
    if category.endswith("_skills"):
        data["skills"][category] = {}
        category_name = category.replace("_skills", "")
        title = None
        for key, val in data["skill_configs"][category_name].items():
          if key.lower() == "title":
            title = val
            break
        data["skills"][category]["name"] = title
        #data["skills"][category]["keywords"] = skills
        data["skills"][category]["keywords"] = list(skills.values())

data['skills'] = list(data['skills'].values())

keys_to_remove = [key for key in data.keys() if key.endswith("_skills")]
# Remove the keys from the JSON object
for key in keys_to_remove:
    data.pop(key)
data.pop("work_tags")
data.pop("skill_configs")
main_json = json.dumps(data, indent=4)

json_filename = '/etc/resume-data.json'
with open(json_filename, 'w') as file:
    file.write(main_json)
#!cat /etc/main_json_output.json

### check if ssh file exists
#output = !ls "/content/drive/MyDrive/resume_v1/private/ssh.zip"
#if "No such file or directory" in output[0]: # no ssh file, clone public repo
print("Hang in tight. This will take a long time of about 5 minutes. You can do other things in the meantime as our engineers work to optimize the blotted code.")
import threading
import time

other_processes_finished = False

def mark_processes_finished():
    global other_processes_finished
    other_processes_finished = True

def display_elapsed_time():
    start_time = time.time()
    while not other_processes_finished:
        elapsed_time = time.time() - start_time
        elapsed_minutes = int(elapsed_time // 60)
        elapsed_seconds = int(elapsed_time % 60)
        print(f"\rElapsed time: {elapsed_minutes} minutes {elapsed_seconds} seconds", end="")
        time.sleep(1)
def long_process():
    !rm -rf vue-resume
    !git clone https://github.com/csymapp/vue-resume.git > /dev/null 2>&1
    !cd vue-resume/ && cp '/etc/resume-data.json' ./
    !cd vue-resume/ && pwd && npm install > /dev/null 2>&1 && npm install postcss@7.0.32 > /dev/null 2>&1 && npm run generate > /dev/null 2>&1
    !wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb > /dev/null 2>&1
    !apt install ./google-chrome-stable_current_amd64.deb > /dev/null 2>&1
    !apt-get install jq > /dev/null 2>&1
    !rm ./google-chrome-stable_current_amd64.deb > /dev/null 2>&1
    !name=$(jq -r '.basics.name' /content/vue-resume/resume-data.json) && google-chrome --headless \
                  --disable-gpu \
                  --disable-software-rasterizer \
                  --disable-dev-shm-usage \
                  --no-sandbox \
                  --user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3" \
                  --print-to-pdf="/content/drive/MyDrive/resume_v1/public/$name - Résumé.pdf" \
                  --hide-scrollbars \
                  file:///content/vue-resume/dist/index.html > /dev/null 2>&1
    mark_processes_finished()

display_thread = threading.Thread(target=display_elapsed_time)
display_thread.start()

other_processes_thread = threading.Thread(target=long_process)
other_processes_thread.start()

other_processes_thread.join()
display_thread.join()
dirId = getoutput("xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/public")
message = f'Pdf resume created. Saved locally in <a target="_blank" href="https://drive.google.com/drive/u/0/folders/{dirId}">your google drive.</a>.'
display(HTML(message))

Mounted at /content/drive


Hang in tight. This will take a long time of about 5 minutes. You can do other things in the meantime as our engineers work to optimize the blotted code.
Elapsed time: 0 minutes 5 seconds/content/vue-resume
Elapsed time: 8 minutes 9 seconds

In [ ]:
#@title renderers
%%shell
echo "abcd" > password.txt
sudo adduser --gecos "" --disabled-password newusername1 < password.txt
sudo usermod -aG sudo newusername1
#cd vue-resume

#npm init -y
#npm install
#npm install resume-cli
#npm install -g jsonresume-theme-elegant --unsafe-perm
#cp resume-data.json resume.json
#wget https://gist.githubusercontent.com/ertugdlk/45c7cccb43e66714a90cdfb8fae5ea56/raw/960ac8a54d1aadf6cbfb68ea547a29d2cb42774d/resume.json -O resume.json
#cp /content/vue-resume/resume-data.json resume.json
#which resume
#which npm
#curl -sL https://deb.nodesource.com/setup_14.x | sudo bash -

#su - newusername1 -c 'export PATH="/tools/node/bin:$PATH" && /tools/node/bin/npm init -y && /tools/node/bin/npm install && /tools/node/bin/npm install -g resume-cli'
#su - newusername1 -c  'export PATH="/tools/node/bin:$PATH" && /tools/node/bin/npm install jsonresume-theme-elegant'
#su - newusername1 -c  'export PATH="/tools/node/bin:$PATH" && /tools/node/bin/npm install jsonresume-theme-actual'
su - newusername1 -c  'export PATH="/tools/node/bin:$PATH" && /tools/node/bin/npm install jsonresume-theme-direct'

#su - newusername1 -c "pwd && cp /content/vue-resume/resume.json ./ && /tools/node/bin/node /tools/node/bin/resume export --format pdf --theme elegant resume.pdf"
su - newusername1 -c "pwd && /tools/node/bin/node /tools/node/bin/resume export --format pdf --theme actual resume-actual.pdf"
#su - newusername1 -c "pwd && /tools/node/bin/node /tools/node/bin/resume export --format pdf --theme flat resume-flat.pdf"
#su - newusername1 -c "pwd && /tools/node/bin/node /tools/node/bin/resume export --format pdf --theme compact resume-compact.pdf"
su - newusername1 -c "pwd && /tools/node/bin/node /tools/node/bin/resume export --format pdf --theme direct resume-direct.pdf"

# Masters
This section is optional and is for more advanced users. It will create a webpage of your resume and get it hosted on github pages.

In [ ]:
#@title Push to github
%%shell
apt-get install jq > /dev/null 2>&1
repo=$(jq -r '.basics.repo' /content/vue-resume/resume-data.json)
if [ -z "$repo" ] || [ "$repo" = "null" ]; then
    file_url="https://docs.google.com/spreadsheets/d/$SHEETID/edit"
    echo "Repo config not found in basics.repo in $file_url"
    exit 0
fi

sshZipped="/content/drive/MyDrive/resume_v1/private/ssh.zip"
privateId=$(xattr -p 'user.drive.id' /content/drive/MyDrive/resume_v1/private)

if ! [ -e "$sshZipped" ]; then
    file_url="https://drive.google.com/drive/u/0/folders/$privateId"
    echo "Compressed ssh key not found in $file_url"
    exit 0
fi

echo -n "Please enter ssh zip password: "
read -s password
echo
export SSH_ZIP_PASSWORD="$password"
cp "$sshZipped" /tmp/ssh.zip
unzip -P "$SSH_ZIP_PASSWORD" -oq /tmp/ssh.zip
rm -rf /root/.ssh
mkdir -p /root/.ssh
rsync -aq ssh/  ~/.ssh/
rsync -aq ssh/   /root/.ssh/
ssh-keyscan GitHub.com > /root/.ssh/known_hosts 2>/dev/null
ssh-keyscan GitHub.com > ~/.ssh/known_hosts 2>/dev/null
chmod 644 ~/.ssh/known_hosts
chmod 600 ~/.ssh/id_rsa
chmod 644 /root/.ssh/known_hosts
chmod 600 /root/.ssh/id_rsa

eval "$(ssh-agent)" > /dev/null 2>&1
echo "$SSH_ZIP_PASSWORD" | ssh-add ~/.ssh/id_rsa > /dev/null 2>&1 || {
    echo "Bad paraphrase. Try again."
    exit 0
}

git config --global user.email "surgbc@gmail.com"
git config --global user.name "Brian Onang'o"

cd /tmp
repoName=$(echo $repo | sed "s|.*/||")
rm -rf "$repoName"
git clone "git@github.com:$repo.git" > /dev/null 2>&1
if ! [ -e  "$repoName" ]; then
    echo "Failed to clone repo. Probably wrong organization/repo($repo) supplied in basics.repo"
    exit 0
fi
cd "$repoName"
cp '/etc/resume-data.json' ./
git add . && git commit -m "$(date)" > /dev/null 2>&1 && git push origin master || {
    echo "Git commands failed. Probably everything is up to date in your resume."
    exit 0
}

Please enter ssh zip password: 
Git commands failed. Probably everything is up to date in your resume.
